##  Python REPL

Python REPL（Read-Eval-Print Loop）是一个交互式编程环境，允许程序员立即执行代码并获取结果。REPL 这个术语来源于描述这种环境工作方式的四个步骤：

- Read（读取） - 读取用户输入的代码。
- Eval（求值） - 对输入的代码进行求值。
- Print（打印） - 打印求值的结果。
- Loop（循环） - 返回第一步，等待用户输入更多的代码。

在 Python 中，REPL 通常是通过命令行工具 python 或 python3（取决于安装的 Python 版本）访问的。启动这个工具后，用户可以直接输入 Python 代码，并立即看到代码执行的结果。


In [1]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

In [2]:
python_repl.run("print(1+3)")

Python REPL can execute arbitrary code. Use with caution.


'4\n'

In [4]:
from langchain_openai import ChatOpenAI, OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.1,
)
llm = model

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

promptFormat = """{query}

请根据上面的问题，生成Python代码计算出问题的答案，最后计算出来的结果用print()打印出来，请直接返回Python代码，不要返回其他任何内容的字符串
"""
prompt = ChatPromptTemplate.from_template(promptFormat)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [17]:
result = chain.invoke({"query":"3箱苹果重45千克。一箱梨比一箱苹果多5千克，3箱梨重多少千克？"})
result

'```python\n# 计算每箱苹果的重量\napples_per_box = 45 / 3\n\n# 每箱梨比每箱苹果多的重量\nweight_difference = 5\n\n# 计算每箱梨的重量\npears_per_box = apples_per_box + weight_difference\n\n# 计算3箱梨的总重量\ntotal_pears = pears_per_box * 3\n\nprint(total_pears)\n```'

In [45]:
def parsePython(codeStr):
    codeStr = result.replace("```python","")
    codeStr = result.replace("```","")
    return codeStr

In [49]:
chain = prompt | llm | output_parser | parsePython
chain

ChatPromptTemplate(input_variables=['query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}\n\n请根据上面的问题，生成Python代码计算出问题的答案，最后计算出来的结果用print()打印出来，请直接返回Python代码，不要返回其他任何内容的字符串\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000190FB262590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001909EB85AE0>, temperature=0.1, openai_api_key=SecretStr('**********'), openai_api_base='http://127.0.0.1:1234/v1', openai_proxy='')
| StrOutputParser()
| RunnableLambda(parsePython)

In [50]:
result = chain.invoke({"query":"3箱苹果重45千克。一箱梨比一箱苹果多5千克，3箱梨重多少千克？"})
result

'\n# 计算每箱苹果的重量\napples_per_box = 45 / 3\n\n# 每箱梨比每箱苹果多的重量\nweight_difference = 5\n\n# 计算每箱梨的重量\npears_per_box = apples_per_box + weight_difference\n\n# 计算3箱梨的总重量\ntotal_pears = pears_per_box * 3\n\nprint(total_pears)\n\n40'

In [51]:
python_repl.run(result)

'60.0\n'

In [52]:
chain = prompt | llm | output_parser | parsePython | python_repl.run
chain

ChatPromptTemplate(input_variables=['query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}\n\n请根据上面的问题，生成Python代码计算出问题的答案，最后计算出来的结果用print()打印出来，请直接返回Python代码，不要返回其他任何内容的字符串\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000190FB262590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001909EB85AE0>, temperature=0.1, openai_api_key=SecretStr('**********'), openai_api_base='http://127.0.0.1:1234/v1', openai_proxy='')
| StrOutputParser()
| RunnableLambda(parsePython)
| RunnableLambda(run)

In [53]:
result = chain.invoke({"query":"3箱苹果重45千克。一箱梨比一箱苹果多5千克，3箱梨重多少千克？"})
result

'60.0\n'